<div style="text-align:center">
<h1>Transforming Data</h1>
<h2>7SSG2059 Geocomputation 2017/18</h2>
<h3 style="font-style:italic">Jon Reades</h3>
</div>

## This Week’s Overview

This week we're going to explore how visualising data (using pandas and seaborn) helps us to make more sense of our data. We're then going to move on to automating this process because coding isn't _just_ about being able to load lots of data, it's also about being able to be constructively lazy with the data once it's loaded.

## Learning Outcomes

By the end of this practical you should:
- Have created a set of different plots using Seaborn
- Have automated the presentation of data for a number of columns
- Have grasped how a mix of graphs and numbers can help you make sense of your data

## Statistics as Judgement

We hope that you'll go on to make lots of use of what you're learning here, but the single most important thing that you can take away from the remaining weeks of the class is the idea that statistics is not _truth_, it is _judgement_.

[![Are you above average?](http://img.youtube.com/vi/hQLCWHww9OQ/0.jpg)](http://www.youtube.com/watch?v=hQLCWHww9OQ)

### Data as Representation of Reality

The statistician George Box [once said](https://en.wikipedia.org/wiki/All_models_are_wrong) "all models are wrong but some are useful". Now you might think that 'wrong' _necessarily implies_ uselessness, but this aphorism is a lot more interesting than that: let's review the idea of statistics as the study of a '[data-generating process](http://onlinelibrary.wiley.com/doi/10.1111/j.1467-9639.2012.00524.x/full)'. 

The data that we work with is a _representation_ of reality, it is not reality itself. Just because I can say that the height of human beings is normally distributed with a mean of, say, 170cm and standard deviation of 8cm doesn't mean that I've _explained_ that process. What I have done is to say that reality can be reasonably well approximated by a data-generating process that uses the normal distribution. 

Given that understanding of height distributions, I know that someone with a height of 2m is very, very improbable. Not impossible, just highly unlikely. So if I meet someone that tall then that's quite exciting! But my use of a normal distribution to represent the empirical reality of human height doesn't mean that I think our height is _actually_ distributed randomly using a gigantic lottery system: some parts of the world are typically shorter, other parts typically taller; some families are typically shorter, while others are typically taller...

So the _real_ reason for someone's height is to do with, I assume, a mix of genetics and nutrition. However, across large groups of people it's possible to _represent_ the cumulative impact of those individual realities with a simple normal distribution. And using that simplified data-generating process allows me to do things like estimate the likelihood of meeting someone more than 2m tall (which is why I'd be excited to do so, though not as excited as the guy in the next video...).

Here's a (genuinely terrifying) video that tries to explain this whole idea in a different way:

[![From reality to make-believe](http://img.youtube.com/vi/HAfI0g_S9oo/0.jpg)](https://www.youtube.com/watch?v=HAfI0g_S9oo)

In the same way, real individuals earn different incomes for all sorts of reasons: skills, education, negotiation ability... and, of course, systematic discrimination or bias. Because of wide variations in individual lived experience, it's quite hard to _prove_ that any _one_ person has been discriminated against unless you have the 'smoking gun' of an email or other direct evidence that this has happened. 

But if I have data on _many_ men and women (from a company, industry, or society) to work with, then I can take a look at what data-generting processes best describe what I've observed. And I can also create a data generating process that would describe what I'd _expect_ to see if no systematic discrimination were taking place at all. I make that sound simple but, of course, it's really hard to do this properly: do you account for the fact that discrimination has probably happened throughout someone's lifetime, not just in their most recent job? And so on.

### What is _Significant_?

But, when we have created a data-generating process that captures this at what we feel is an appropriate level, then the analytical process becomes about testing to see if there is a _significant_ difference between what I expected and what I observed. Once we've done that, then we can start to rule out claims that 'there are no good candidates' and the other defences of the indefensible. It is always theoretically _possible_ that a company had trouble finding qualified candidates, but as you put together the evidence from your model it may well  become increasingly _improbable_.

So, always remember that, while the data is not reality, it _is_ a very useful abstraction of reality that allows us to make certain claims about what we expected to see. Linking our observations to what we know about the characteristics of the data generating process then allows us to look at the _significance_ of our results or to search for outliers in the data that seem highly improbable and, consequently, worth further investigation.

In [ ]:
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import pysal as ps
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.cm as cm
import urllib
import zipfile
import re
import os

import random
random.seed(20171107)
np.random.seed(20171107)

def dld_merge_gsa_data(dst='merged.csv.gz'):
    """
    Function to automatically build the merged LSOA and air quality data set.
    
    Keyword arguments:
    dst -- a local path to use as a destination (default merged.csv.gz)
    """
    dst_compression='gzip'
    
    # Check if local copy exists...
    if os.path.exists(dst):
        # Yes, then just read and return the data frame
        print("File already downloaded.")
        # The 'low memory' option means pandas doesn't guess data types
        df = pd.read_csv(dst, compression=dst_compression, low_memory=False)
        
    else:
        # No, then download and save a local copy
        print("Downloading remote data...")
        
        # It is possible to have a remote file with no
        # compression so this is just a piece of forward
        # looking code....
        df1 = pd.read_csv(
            'https://github.com/kingsgeocomp/geocomputation/blob/master/data/LSOA_AirQuality.csv.gz?raw=true',
            compression='gzip', low_memory=False
        ) # The 'low memory' option means pandas doesn't guess data types
        df2 = pd.read_csv(
            'https://github.com/kingsgeocomp/geocomputation/blob/master/data/LSOA%20Data.csv.gz?raw=true',
            compression='gzip', low_memory=False
        )
        df = pd.merge(df1, df2, how='left', on='LSOA11CD')
        # And save it
        print("Writing to local file...")
        df.to_csv(dst, compression=dst_compression, index=False)
    print("Done.")
    return df

In [ ]:
df = dld_merge_gsa_data()
df.set_index(['LSOA11CD'], drop=True, inplace=True)

----

# Transformation

Transformations are useful when a data series has features that make comparisons or analysis difficult, or that affect our ability to intuit meaningful difference. By manipulating the data using one or more mathematical operations we can sometimes make it more *tractable* for subsequent analysis. In other words, it's all about the _context_ of our data.

[![How tall is tall?](http://img.youtube.com/vi/-VjcCr4uM6w/0.jpg)](http://www.youtube.com/watch?v=-VjcCr4uM6w)

### Working with Demonstration Data

Here's an example from one of our earlier lessons, but now written as Python code: let's say that we want to understand how student heights (and the visiting Mr. G.) are distributed within a class. It's not at all easy if all you have to go on is a list of raw heights: 160cm, 158cm, 150cm, 185cm, 172cm, 175cm, 166cm... and, of course, Mr. Bill Gates himself, to know how big a range you've got on your hands and where people fall relative to the mean.

Let's try writing this out as Python code:
```python
# Create an empty data frame to 
# hold our height data
df2 = pd.DataFrame() 
```
Here we've created an empty data frame – as yet it contains no data.
```python
# Create and add a series
df2['Height'] = pd.Series(
    [160, 158, 150, 185, 172, 175, 166, 168],
    index = ['Judy','Frank', 'Alice', 'Eve', 'Bob', 'Carlos', 'Dan', 'Bill G.']
)
```
Then we create a new data series: the data for the series is the heights, the index is the name of the student. We assign this new data series to 'Heights' in the data frame.
```python
# Look at the results
df2.describe()
```
And, as always, a good next step is to check that you got what you expected.

In [ ]:
df2 = pd.DataFrame() 

# Create and add a series
df2['Height'] = pd.Series(
    [160, 158, 150, 185, 172, 175, 166, 168],
    index = ['Judy','Frank', 'Alice', 'Eve', 'Bob', 'Carlos', 'Dan', 'Bill G.']
)

# Look at the results
df2.describe()

Now let's add their wealth...

In [ ]:
df2['Wealth'] = pd.Series([28300, 21258, 37234, 32748, 18536, 75093, 124382, 5124398742348], index=df2.index)

# Check the results
df2.describe()

An obvious first step to understanding this student data would be to use the mean ($\mu$) from the data since that tells us the _average_ height and wealth of all students in the class. But wouldn't it be even _more_ interesting to be able to look at how tall students are _relative_ to the mean? Is there someone from the basketball team taking this course? Or maybe the cox from a crew? How could we make it easy to compare the difference between each student and the overall class average in order to spot these 'special cases'?

In many cases, the best way to make this comparison is to _subtract the mean_. Why is that? What does it achieve?

Let's think it out:
1. If a student is shorter _than average_ then their transformed height is less than 0
2. If a student is taller _than average_ then their transformed height is more than 0
3. The distance from 0 (e.g. -20 vs -3) gives us _some_ sense of how short or how tall someone is relative to that mean

In a mathematical form we'd write this transformation as:
$$
x - \mu
$$

In pandas we can express this transformation as:
```python
df['<new column name>'] = df.<column>-df.<column>.mean())
df.describe()
```

We can break this apart as:

* `df.<column>` – this is the _entire_ data series
* `df.<column>.mean()` – this calculates the mean ($\mu$) of the data series
* We perform this calculation and then use the results to create a new data series
* We assign this series to a new column in the data frame called `<new column name>`.

Pandas is smart enough to know that it needs to take _each_ student height and then subtract the mean height of all students from that value. So even though it looks like we're performing a single calculation, we're actually performing as many calculations as there are rows in the data frame but without needing to write any tricky code!

To recap: looking at the heights of the students (whether in code or in the notebook generally) it's hard to tell how far each student is from average, and who might be especially (*significantly*) tall or short. When this happens we can _transform_ the raw data in order to make it easier to see and interpret this variation.

*Remember*: subtracting the mean is a linear transformation (unlike the log-transform).

Try the transformation in the coding area below:

In [ ]:
df2['TransformedHeights'] = df2.Height-df2.Height.mean()
df2['TransformedWealth']  = df2.Wealth-df2.Wealth.mean()
df2.describe()

What do the Transformed Heights mean? Does the notation make any sense to you? It may help to remember that Python, like nearly all programming languages, can rack up very, very minute errors when working with floating point numbers.

So the output is in something called scientific notation, where numbers are represented using an exponent for accuracy and consistency in the formatting. To change this, try a Google search on `"python data frame force non-scientific output"`. You should find a solution in the first few results. 

The concept of `lambda` is something we haven't seen before: previously, when we wanted to define a function we _had_ to use `def <function name>:`, but sometimes we just need a tiny snippet of code to do something useful in a function-like way and don't want to have to write a full definition. That's where `labmda` comes in: it is creating something called an *anonymous* function, meaning that we can define a function without giving it a name. Why is this useful? [Read more about lambda](https://pythonconquerstheuniverse.wordpress.com/2011/08/29/lambda_tutorial/).

## Selecting the Right Transformation

Now let’s have a look at our real-world data (or should that be "The Real World"?). We've got quite a few columns to work with, but they describe quite different types of phenomena. Let's see how this plays out in the distributions and, consequently, the types of transformations that area appropriate...

In [ ]:
# Add comments to explain what this function
# is doing.
def normal_from_dist(series): 
    mu = series.mean()
    sd = series.std()
    n  = len(series)
    s = np.random.normal(mu, sd, n)
    return s

# Now use the function above to plot both a distribution plot 
# with both histogram and KDE, and then add a _second_ overplot
# distplot to the same fig. The overplot should not have a histogram
# and the color should be red. You should be able to check the 
# last figure (Owned) against the next block of questions to see
# if you got the right answer.
for c in ['RoadsArea','NOxmax','Owned']:
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)
    sns.distplot(???)
    sns.distplot(normal_from_dist(???), hist=???, color='red')

**Q1**: For which of the columns are the mean and standard deviation _most_ appropriate as measures of centrality and spread?

**A1**: _Your answer **and rationale** here._

## Making use of a transformation

Let's focus on the 'Owned' (_i.e._ home ownership) column. Without allowing for the fact that different areas have different numbers of households to begin with (we'll get to _that_ later) how do we go about setting a threshold for areas with high concentrations of home ownership? Let's look at the chart again (also a good way to check you got the right code in the block above).

![Ownership Distribution](https://github.com/kingsgeocomp/geocomputation/blob/master/img/Owned.png?raw=true)

We _know_ that the middle is somewhere about 350 (we can get the exact number using `<series>.describe()`) but is 400 extreme? What about 600? Or 500? 

#### The Standard Normal

The standard normal distribution has some useful properties; here's a refresher:

![Standard Normal](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Boxplot_vs_PDF.svg/500px-Boxplot_vs_PDF.svg.png)

The cut-offs that we're usually the most interested in are:

| Confidence Level | z\*-value |
|------------------|-----------|
| 50% | 0.675 |
| 80% | 1.28 |
| 90% | 1.645 (by convention) |
| 95% | 1.96 |
| 98% | 2.33 |
| 99% | 2.58 |
| Outlier | 2.698 | 

What you'll notice is that we can have _any_ cut-off we want. By convention the 95% confidence interval had become the standard for a 'significant' result, but over the past 10-15 years the importance of publications for researchers (and the fact that reviewers were using 95% CIs as an _heuristic_ for determining whether the research had 'succeeded') led the rise of _p-hacking_ (or _p-value hacking_) in which authors basically played with their data until their results surpassed this magical threshold. As the starting example shows, you can say _anything_ with data if you want.

<div style="text-align:center">
<h3>“If you torture the data long enough, it will confess.”</h3>
<h4>― Ronald H. Coase, Essays on Economics and Economists</h4>
</div>

### Using the transformation to select 'outliers'

Using common sense, the 95% CI applied to the `Owned` data series, and a mix of code we've used above _and_ in earlier weeks:
1. How many LSOAs are _above_ the upper CI?
2. How many LSOAs are _below_ the lower CI?
3. What are the LSOA names for the first three LSOAs just _above_ the upper CI?
4. What are the LSOA names for the first three LSOAs just _below_ the lower CI?
5. Can you plot the extreme values on the same plot?

The output I'm looking for is below, but I'll settle for anything _close_ (_i.e._ you have the right results even if the formatting if off) and I've included some comments and snippets of code to get you started...

In [ ]:
# First, we need the mean and standard deviation of the data
mu = df.Owned.???
sd = df.???

# Then we need our upper and lower CIs
uci = mu + 1.96 * sd
lci = mu - 1.96 * sd

# Then we need to select the LSOAs falling above
# and below our CI values (you can sort the results
# at the same time, or do this as a separate command)
udf = df.loc[df.Owned > uci].sort_values(by='Owned', ascending=True) # udf == upper df
ldf = df.loc[???].sort_values(by='Owned', ascending=???) # ldf == lower df

# Output main results
print("There are {0} LSOAs ({1:.2f}%) above the upper CI of {2:.2f}".format(len(udf),(float(len(udf))/len(df))*100,uci))
print("There are {0} LSOAs ({1:.2f}%) below the lower CI of {2:.2f}".format(???))
print("The upper LSOAs are: " + ", ".join(udf.LSOA11NM.head(3).values))
print("The lower LSOAs are: " + ", ".join(ldf.LSOA11NM.head(3).values))

# And plot the results
fig1 = plt.figure()
plt.title("Ownership Extremes")
ax1 = fig1.add_subplot(111)
sns.distplot(???, color='green') # ???
sns.distplot(???, color='red') # ???

Here's what I got:
```shell
There are 62 LSOAs (1.28%) above the upper CI of 591.19
There are 62 LSOAs (1.28%) below the lower CI of 60.81
The first three upper LSOAs are: Sutton 009A, Richmond upon Thames 011D, Richmond upon Thames 012D
The first three lower LSOAs are: Kensington and Chelsea 005D, Lambeth 009B, Tower Hamlets 018A
```

![Ownership Extremes](https://github.com/kingsgeocomp/geocomputation/blob/master/img/Owned_Extremes.png?raw=true)

**Q2.** Can you think why we only seem to have 2.56% of the LSOAs falling outside of our 95% CI? Shouldn't it be 5%?

**A2.** _Your answer here_

**Q2 (Bonus Part 1).** Can you think how to get a count of LSOAs by borough for each of the two extreme ends of the distribution? Write your code below. _Hint:_ You want to look at the highest-voted answer on [this page](https://stackoverflow.com/questions/19384532/how-to-count-number-of-rows-in-a-group-in-pandas-group-by-object).

**Q2 (Bonus Part 2).** What do you notice about these upper and lower outliers? Is there a geographical pattern to suggest that this might not be quite what you want?

In [ ]:
print("Upper outliers: ")
print(udf[['LAD11NM']].???)
print("\nLower outliers: ")
print(ldf[['LAD11NM']].???)

## Logarithmic Transformation

Logarithmic transformations are also considered fairly simple, but they are _non_-linear transformations and so they do change the relationships in your data in important ways. Although you _could_ use any logarithm, the natural log is considered the most useful since both the mean and standard deviation retain _some_ meaning (though you probably wouldn't report these as such). If you don’t remember what a logarithm is try these:

[![From reality to make-believe](http://img.youtube.com/vi/zzu2POfYv0Y/0.jpg)](https://www.youtube.com/watch?v=zzu2POfYv0Y)

[![From reality to make-believe](http://img.youtube.com/vi/akXXXx2ahW0/0.jpg)](https://www.youtube.com/watch?v=akXXXx2ahW0)

[![From reality to make-believe](http://img.youtube.com/vi/0fKBhvDjuy0/0.jpg)](https://www.youtube.com/watch?v=0fKBhvDjuy0)

The last video was made by Ray & Charles Eames, two of the 20th Century’s most famous designers.

**Q3.** Why are log-transforms considered non-linear?

**A3.** _Your answer here_

## Let's imagine...

Let's imagine that:

1. I think that there is relationship between pollution and the presence of major roads.
2. I don't know what kind of relationship there might be.

We're going to start with the simplest approach: a chart!

In [ ]:
g = sns.jointplot(x=df.RoadsArea, y=df.NOxmax, size=6)
g.fig.suptitle('NOx (Max) against Roads Area"', fontsize=18,color="r",alpha=0.5)

Hmmm, that didn't help very much. It looks like our RoadsArea data is very heavily skewed and that there are a lot of very low values in the data. There also seems to a be at least one major outlier that is _so_ different in scale that I'd want to know whether to even keep it in the analysis -- it looks like what would be called a 'leverage point' in a regression model: something that is so 'out of whack' that it alters the entire regression! 

Let's try stripping that out and running this code again:

In [ ]:
print("The outlier LSOAs are: " + ",".join(df[df.NOxmax > 4000]['LSOA11NM'].values))
df = df[df.NOxmax < 4000]

g = sns.jointplot(x=df.RoadsArea, y=df.NOxmax, size=6)
g.fig.suptitle('NOx (Max) against Roads Area"', fontsize=18, color="r",alpha=0.5)

Quite a change right? Notice that the correlation has dropped markedly and that the new chart presents a much more complex picture of NOx and Roads Areas. 

Regardless, on the whole in this presentation it's _still_ really hard to tell what's going because of the heavy skew in the Roads data... Perhaps a different transformation might help?

### Logarithmic Transforms in Pandas

To create a new series in the data frame containing the natural log of the original value it’s a similar process to what we've done before, but since pandas doesn't provide a log-transform operator (i.e. you can’t call `df.Owned.log()` ) we need to use the `numpy` package:
```python
series = pd.Series(np.log(df.RoadsArea))
```
Try performing the transformation and then `describe()` the results in the coding area below. Is it more clear to you now why a log-transform is a non-linear transformation?

**_Warning_**: What is the natural log of 0? And of 1? What do you need to change in the code above to make this transformation work? Output below...

In [ ]:
# Why do I do this here???
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x))

series = pd.Series(???)
print(series.describe())
sns.distplot(series, color='green')

```
count    4834.000000
mean        8.689413
std         4.910561
min         0.000000
25%         7.411466
50%        11.244498
75%        11.920692
max        14.769750
Name: RoadsArea, dtype: float64
```
Now this _is_ interesting: the output of the graph shows what seems to be two quite different things going on in our data! We've obviously got the LSOAs that contain _no_ major roads, but then we've got something else that is _much_ closer to 'normal' (though obviously not properly normal as there is clear evidence of negative skew). Technically, this is closer to _log-normal_ and you **should not** think of the mean or standard deviation as being numbers that you would report. What they _can_ do is help you to set a cut-off for the LSOAs that are most impacted by major roads...

In [ ]:
hrds = df.loc[df.RoadsArea > 0] # hrds == has (major) roads

g = sns.jointplot(x=np.log(hrds.RoadsArea), y=np.log(hrds.NOxmax), size=6)
g.fig.suptitle('Log-Transformed NOx (Max) against Roads Area"', fontsize=18,color="r",alpha=0.5)

There still isn't a clear answer here, but at this point I would be thinking: there's _something_ going on, but looking at the correlation coefficient (Pearson's r == 0.26 [_stay tuned for Week 9!_]) I wouldn't feel very comfortable that NOx (Max) values were predicted solely by the total area of a LSOA within 250m of major roads... this is what we mean by building a model: we know we've got our hands on something useful, but we also know that we need to work on it some more to make the model _useful_.

## Standardisation

### Proportional Standardisation

Clearly, a proportion (e.g. a percentage) of the total count contained in one LSOA is one way of standardising data since, unless you're measuring change, it limits the range to between 0% and 100%. Programmers and statisticians almost always write a proportion in a decimal format so the range is between 0.0 and 1.0. 

Mathematically, however, the notation is a little more forbidding:
$$
p_{i} = \frac{x}{\sum_{i=1}^{n} x}
$$
However, it's important that you begin to familiarise yourself with the mathematical notation since many papers on computational geography will make use of this form. Let's break it down:

1. It's a fraction: the observation _x_ divided by, errrr, something involving x
2. The numerator is easy
3. The denominator is hard
4. The key is in the $i=1$ and $n$, which tells us that the sum is for 'all i' (i.e. summing for every _x_-value in the data set) 

In other words, we take the `sum()` of the column of `x` as the divisor for *each* observation of `x`. 

To put it in terms of our heights data:

1. Take each person's height (which we call '_x_')
2. Add up (sum: $\sum$) every _x_ (which runs from 1.._n_) in the data set
3. And divide \#1 by \#2

But the point is that that notation applies for every data set, not just our heights data. That's where the mathematical formula is more useful – it's not linked to the specifics of this particular data set.

### Proportional Standardisation in Pandas

We can calculate the proportion for each area using a similar approach to what we've seen before in the Transformation section:
```python
df['proportion'] = df.<column>/df.<column>.sum()
```
You can see the link here between the mathematical notation and the computational representation, right?

Let's do this for the columns that we've been working with and try printing out various summary metrics for the new column and comparing them to the raw values. 

In [ ]:
dfs = df.copy() # Copy df to df-standardised

for c in ['RoadsArea','NOxmax','Owned']:
    dfs[c] = ??? # Note that we're overwriting the column in dfs _only_
    print(dfs[c].describe())
    
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)
    sns.distplot(dfs[c]) 
    sns.distplot(normal_from_dist(???), hist=False, color='red')

### Think!

In this case are the simple standardisations very useful? Can you think why we might not be _that_ interested in these results? So if that's _not_ what we want, then what _do_ we want? It's worth thinking about what these columns are actually measuring:

1. Roads Area is a measure the area (in m^2) that are near to a major road. What other column contains areal data that might be relevant?
2. Owned is the number of properties that are owned outright. What other column contains information about the total number of potential properties (allowing for some fudging for houses that are owned by one household but shared with others).

### Areal Standardisation

So the most basic and obvious standardisation at this point would be to work out what the 'Roads Area' is _really_ measuring -- we've got a total area near roads but still have no idea how big the actual LSOA _is_. So it's quite possible that an outer borough has a lot of area near major roads, but a low NOx reading overall because it also has a lot of areas that are a _long_ way from major roads and their pollutants.

Try copying the code block above into the block below and adjusting it for the Roads Area data so that it gives you a more useful answer. *Hint*: the right answer in _this particular case_ will have the following output:
```python
count   4,834.0000
mean        0.4224
std         0.3659
min         0.0000
25%         0.0086
50%         0.3937
75%         0.7705
max         1.0000
dtype: float64
```

In [ ]:
dfs = df.copy()

for c in ['RoadsArea']:
    series = ???
    print(series.describe())
    
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)
    sns.distplot(series)
    dfs[c+'Std'] = series

### Household Standardisation

Let's do the same for the ownership data. Here we wouldn't standardise by area (we've already got a population density measure anyway) but by something else that would give us a sense of the _density of ownership_ in an area...

Here's your hint:
```python
count   4,835.0000
mean        0.4988
std         0.2252
min         0.0095
25%         0.3181
50%         0.4880
75%         0.6689
max         0.9740
dtype: float64
```

In [ ]:
# dfs = df.copy() # This would overwrite the copy created above

for c in ['Owned']:
    series = ???
    print(series.describe())
    
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)
    sns.distplot(???)
    dfs[???+'Std'] = series # Std == Standardised

Right, so now we see a range of densities for ownership between 0.0 and something just south of 1.0. So ownership varies dramatically by LSOA, though the mean is about 50%. Let's compare how our standardised data compares with the original data... I get a plot that tells me Pearson's _R_ is 0.9.

In [ ]:
sns.jointplot(x=dfs.OwnedStd, y=dfs.Owned)

So yes, there is a very strong relationship between the raw count of home-owning households and our estimation of the density of home-ownership, but it's _also_ not perfect. Let's see if we can dig into this further to see what has been most affected.

## Z-Score Standardisation

The z-score is a common type of standardisation, but it's a little more complex than a simple proportion; however, both are designed to enable us to compare data across different groups of observations. We can easily compare two percentages to know which one is more, and which less (e.g. I got 80% on one exam and 70% on the other).

But let's think about a slightly different question: 'which exam did I do _better_ on?' What if you got 80% on an exam where everyone else got 85%? Suddenly that doesn't look quite so good right? And what if your 70% was on an exam where the average mark was 50%? Suddenly that looks a lot better, right?

The z-score is designed to help you perform this comparison directly.

As a reminder, the z-score looks like this:
$$
z=(x-\mu)/\sigma
$$

That's: `(<data> - <mean>)/<standard deviation>`

### Using the Z-Score

Let's start to bring the idea of the 'data generating processes' to life. The first thing to do with the z-score is to look at what it implies:

1. Subtracting the mean implies that the mean _is a useful measure of centrality_: in other words, the only reason to subtract the mean is if the mean is _meaningful_. If you are dealing with highly skewed data then the mean is not going to be very useful and, by implication, neither is the z-score.
2. Dividing by the Standard Deviation implies that this _is a useful measure of distribution_: again, if the data is heavily skewed or follows some exponential distribution then the standard deviation is not going to very useful as an analytical tool.

So the z-score is _most_ relevant when we are dealing with something that look vaguely like a standard normal distribution (which has mean=0 and standard deviation=1). In those cases, anything with a z-score more than 1.96 standard deviations from the mean is in the 5% significance zone. 

But remember: we can't really say _why_ one particular area has a high concentration of well-off individuals or why one individual is over 2m tall. All we are saying is that this standardised value is a pretty unlikely outcome _when compared to our expectation that people are randomly distributed across the region_ or _that people have randomly-distributed heights of mean 'x' and standard deviation 'y'_. 

Of course, we _know_ that people aren't randomly distributed around the country in the same way that we know that height isn't genuinely random becuase of the influence of genetics, nutrition, etc. But we need a way to pick out what counts as _**significant**_ over- or under-concentration (or height) from the wide range of 'a bit more' or 'a bit less' than 'normal'. **_If_** a normal distribution does a good job of representing the overall distribution of heights (_whatever_ the reason) then someone of 2m is highly unlikely but we can't say _how_ unlikely until we've placed them on the distribution.

Let's put it another way:
* Is 10% of wealthy individuals in a small area a high concentration?
* How about 20%?
* 30%?

The only way to answer that question is to use something like the z-score since it standardises all of the values _against the average_. If wealthy people were distributed at random then we would _expect_ that most areas would have about the average concentration. Some areas will have a few more. Some areas a few less. But according to the way that the standard normal distribution works, _nowhere_ should have a z-score of 10. Or 20, since that is 20 standard deviations from the mean and just shouldn't exist in the lifetime of the universe. So if we see that kind of result then we know two things:

1. That our assumption that normal distribution is a reasonable representation of reality breaks down at some point.
2. That there are _some_ areas with _highly significant_ over- or under-representation by wealthy residents.

But that's ok, because we're trying to set an expectation of what we think we'll see so that we can pick out the significant outliers.

### Setting Expectations

And here we get to the crux of the issue, most frequentist statistics boils down to this: subtracting **what you expected** from **what you got**, and then dividing by **some measure of spread** to control for the range of the data. We then look at what's _left_ to see if we think the gap between expectations and observations is _meaningful_ or whether it falls within the range of 'random noise'.

It should be obvious that it's the _**expected**_ part of that equation that is absolutely crucial: we come up with a process that generates data that _represents_ the most important aspects of reality, but we shouldn't imagine that our process has explained them. It's the first step, not the last.

### Illustrating Z-Score Selection

So let's see how this works... let's take two of the NS-SeC groups: Group 1 and Group 4. And let's investigate their distributions and whether they contain obvious outliers (extremes that shouldn't exist if they were to follow a normal – which is to say: random – distribution). But we _can't_ do this with the raw counts because the raw counts are truncated (you can't have -1 Group 1 households, only positive counts). We'll use the z-score:

### Z-Score Standardisation in Pandas

Let’s take our ownership column and calculate a new series called `OwnedZStd`!

One way to do this is would be to do it in two stages: subtract the mean to create a new series, then divide by the standard deviation into another new series (_note:_ `<column>` is just a generic name; e.g. substitute 'Owned' for column to create actual code):
```python
dfs['<column>LessMean'] = dfs.<column> - dfs.<column>.mean()
dfs['<column>ZStd']     = dfs.<column>LessMean / dfs.<column>.std()
```
Let's be clear: that solution is _perfectly acceptable_ and _perfectly accurate_. It is also, however, less elegant since it creates extra columns of data that are really only temporary variables that don't need to be added to the data frame.

By now you've seen how we can 'chain' together method calls, then you should know that we could _also_ do it this way:
```python
df[<column>] = 
     (df.<column> - df.<column>.mean()) / df.<column>.std()
```
Do you see how we can begin to build increasingly complicated equations into the process of creating a new data series?

### Using the z-score to select outliers

Let's try finding outliers in the raw and standardised data series that we creates above using z-scores. We'll **use the 90% Confidence Interval equivalent** (have a look at the table above to find out the right z-score to use). Below that is some quite complex code that I'll talk you through once you've figured out how to do the z-score selection.

You should get the following results:
```python
Using raw data we got: 10.0124%
Using standardised data we got: 9.5987%
Raw outliers: 
          LAD11NM  count
4         Bromley     46
27      Southwark     40
15       Havering     37
29  Tower Hamlets     35
11        Hackney     34
2          Bexley     27
5          Camden     20
7         Croydon     19
10      Greenwich     16
21        Lambeth     16
```

In [ ]:
# Outliers according to raw scores
raw_outliers = dfs[np.abs( (??? - ???)/??? ) > 1.645]

# Outliers according to standardised scores
std_outliers = dfs[???]

# What percentage of observations did we select?
print("Using raw data we got: {0:.4f}%".format( 100 * (float(len(raw_outliers))/len(dfs)) ))
print("Using standardised data we got: {0:.4f}%".format( 100 * (float(len(std_outliers))/len(dfs)) ))

# And how are they distributed geographically by borough?
# Explanation of this code is below...
print("Raw outliers: ")
print(pd.DataFrame({'count': raw_outliers[['LAD11NM']].groupby('LAD11NM').size()}).reset_index().sort_values(by='count', ascending=False).head(10)) # ???

print("Standardised outliers: ")
print(pd.DataFrame(???) 

OK, so what's with the lines of code that report the results? Let's break it down:
```python
pd.DataFrame({'count': raw_outliers[['LAD11NM']].groupby('LAD11NM').size()}).reset_index().sort_values(by='count', ascending=False).head(10)
```

Let's start by reformatting this to make it easier to read:
```python
pd.DataFrame(
    {'count': raw_outliers[['LAD11NM']].groupby('LAD11NM').size()}
).reset_index().sort_values(by='count', ascending=False).head(10)
```

And now, working from the easy parts to the complex parts:
1. You can see that `pd.DataFrame(...)` is creating a new data from something to do with the `raw_outliers`.
2. If you carefully match the parentheses then you can see where the `(...)` ends:
   * `pd.DataFrame(...).reset_index().sort_values(by='count', ascending=False).head(10)`
   * Let's skip `reset_index` for a minute...
   * `sort_values(by='count', ascending=False)` is something you've seen before, but you might be wondering where 'count' came from since that's not a column in `dfs`
   * `head(10)` is obvious, and we've just tacked it on to the output of the `sort_values` method, so hopefully this makes a bit of sense as take the 10 largest values of `count`.
3. If you look closely you'll see that there is a `count` also inside the `pd.DataFrame()` method call to create a new data frame
   * `{'count': ...}` is the syntax for creating a column named 'count' in a new data frame using the dictionary syntax!
   * `raw_outliers[['LAD11NM']].groupby('LAD11NM').size()` is the name syntax that we saw earlier for creating a count of LSOAs by borough that fell into the upper and lower outliers using standard deviations...
4. The final bit of magic is the `reset_index()` which takes the results of the `groupby` function and turns the whole thing into a new data frame that no longer has the information about the original LSOAs.

I couldn't remember how to do this myself, so I started with a Google search on `convert groupby to dataframe` and then built the code from there...

You'll notice that, while there is overlap between the two reports there are also major differences. I would argue that the second output (from the standardised data) is more _meangingful_ because we've controled for the fact that LSOAs are of different sizes and have different populations. Here's a quick way to drive home the differences:

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
v1 = sns.violinplot(x='LAD11NM', y='Owned', data=dfs, ax=ax)
for item in v1.get_xticklabels():
    item.set_rotation(90)

fig, ax = plt.subplots(figsize=(15,5))
v2 = sns.violinplot(x='LAD11NM', y='OwnedStd', data=dfs, ax=ax)
for item in v2.get_xticklabels():
    item.set_rotation(90)

## Robust Transformations (_e.g._ IQR)

You will recall that we can use the IQR to standardise data where there are outliers. Since the IQR is _robust_, this qualifies as a robust transformation or, as the Python library sci-kit learn (a.k.a. _sklearn_) calls it: a [Robust Scaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html). We can use it together with a transformation to handle something like this:

In [ ]:
dfs['NOxmaxT'] = ??? # Log-transform the NOxmax column

fig, ax = plt.subplots(figsize=(15,5))
sns.distplot(dfs.NOxmax)

fig, ax = plt.subplots(figsize=(15,5))
sns.distplot(dfs.NOxmaxT)

The transformed NOx-max is clearly still not normal, so the z-score would be a poor choice of standardisation. Let's try using sklearn instead:

In [ ]:
from sklearn.preprocessing import RobustScaler

# Try changing the quantile ranges and seeing how this plot changes...
rs = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True) 
dfs['NOxStd'] = rs.fit_transform(dfs[['NOxmaxT']])

fig, ax = plt.subplots(figsize=(15,5))
sns.distplot(???) # What are we plotting here?

You can see how this is still, fundamentally the _same_ data, we've just mucked about with its position and scale on the axis! If you _really_ care about normality, then there's a [Box-Cox transform](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html) in `scipy.stats` to get something that is almost _exactly_ normal.

**Seeing how it works**: sklearn also has a really nice illustration of the different scalers and their effect on a data set. This is _well_ worth a read as it shows how massively the shape of the data can be impacted by transformation and standardisation.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5), sharex=True, sharey=True)
sns.regplot(ax=ax, x=dfs.RoadsAreaStd, y=dfs.NOxStd, scatter_kws={"s":10, "color":"r"})

**Q4**. How does this last plot change your understanding of the relationship (if any) between roads and NOx pollution?

**A4**. _Your answer here_

### Another exploration

Right, so that was one pair of variables. I'd like you to tackle the same kind of work using _either_ (or _both_, if you have time) of the following pairs:

1. Owned and White
2. PrivateRented and Asian

You're free to use others if you really want, but these two should have interesting relationships that you can explore and expand on using a combination of transformation and standardisation.

## Reflection

This is quite a lot of content, so I'd strongly suggest that you take a little while to _think_ about what you've done and how this has helped you to tease out subtle relationships between variables with greater ease. We've 'tortured' the data (up to a point) and it has 'confessed', but it's now up to us to make sense of whether that confession was forced or has actually reveled something meaningful...